# Importing the libraries

In [ ]:
import numpy as np
import random
import os
import tensorflow as tf
from tensorflow import keras
import platform
from tensorflow.keras import layers, optimizers, losses, models, Input, Model
import time # Per misurare il tempo di training
from tensorflow.keras.callbacks import EarlyStopping # Per l'early stopping
import matplotlib.pyplot as plt
from tqdm import tqdm # Per mostrare una barra di progresso

# Set seeds for random operations.

In [ ]:
# --- 1. Impostazione del Seed Globale all'inizio del tuo script ---
# Questo è il punto chiave per la riproducibilità di TUTTO ciò che segue.
MASTER_RANDOM_SEED = 42
np.random.seed(MASTER_RANDOM_SEED)
random.seed(MASTER_RANDOM_SEED) # Imposta anche il seed per la libreria 'random' di Python se la usi
tf.random.set_seed(MASTER_RANDOM_SEED)
os.environ['PYTHONHASHSEED'] = str(MASTER_RANDOM_SEED) # Per operazioni basate su hash (es. ordine dei dizionari)
os.environ['TF_DETERMINISTIC_OPS'] = '1' # Forza operazioni deterministiche in TensorFlow 2.x

# Print the HW Specs.

In [ ]:
print("--- Dettagli dell'Architettura Hardware della Sessione Colab ---\n")

# --- 1. Dettagli CPU ---
print("--- Dettagli CPU ---")
!lscpu
print("\n")

# --- 2. Dettagli RAM (Memoria) ---
print("--- Dettagli RAM (Memoria) ---")
!cat /proc/meminfo | grep MemTotal
print("\n")

# --- 3. Dettagli Spazio su Disco ---
print("--- Dettagli Spazio su Disco ---")
!df -h /
print("\n")

# --- 4. Dettagli Acceleratore Hardware (GPU/TPU) ---
print("--- Dettagli Acceleratore Hardware (GPU/TPU) ---")
try:
    tpu_address = os.environ.get('COLAB_TPU_ADDR')
    if tpu_address:
        resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu=tpu_address)
        tf.config.experimental_connect_to_cluster(resolver)
        tf.tpu.experimental.initialize_tpu_system(resolver)
        print(f"Tipo Acceleratore: TPU (indirizzo: {tpu_address})")
        print("Dispositivi TPU disponibili:")
        for device in tf.config.list_logical_devices('TPU'):
            print(f"  - {device.name}")
    else:
        gpus = tf.config.list_physical_devices('GPU')
        if gpus:
            print(f"Tipo Acceleratore: GPU")
            for gpu in gpus:
                print(f"  - Dispositivo GPU rilevato: {gpu.name}")
            print("\nDettagli GPU specifici (da `!nvidia-smi`):")
            !nvidia-smi
        else:
            print("Tipo Acceleratore: Nessuna GPU o TPU rilevata (in uso CPU)")

except Exception as e:
    print(f"Si è verificato un errore durante la rilevazione dell'acceleratore: {e}")
    print("Tentativo di rilevare i dispositivi TensorFlow standard:")
    devices = tf.config.list_logical_devices()
    if devices:
        for device in devices:
            print(f"  - Dispositivo rilevato: {device.name}, Tipo: {device.device_type}")
    else:
        print("Nessun dispositivo TensorFlow rilevato.")

print("\n--- Analisi Dettagli Hardware Completata ---")

--- Dettagli dell'Architettura Hardware della Sessione Colab ---

--- Dettagli CPU ---
Architecture:             x86_64
  CPU op-mode(s):         32-bit, 64-bit
  Address sizes:          46 bits physical, 48 bits virtual
  Byte Order:             Little Endian
CPU(s):                   2
  On-line CPU(s) list:    0,1
Vendor ID:                GenuineIntel
  Model name:             Intel(R) Xeon(R) CPU @ 2.20GHz
    CPU family:           6
    Model:                79
    Thread(s) per core:   2
    Core(s) per socket:   1
    Socket(s):            1
    Stepping:             0
    BogoMIPS:             4399.99
    Flags:                fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge m
                          ca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht sysc
                          all nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xt
                          opology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq
                           ssse3 fma cx16 pcid sse4_1 sse4

# Connect To Gdrive to store the datasets created.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Define the paths

In [ ]:
# Percorsi dataset
paths = {
    #"0-20": {
    #    "train": "/content/drive/MyDrive/GitHub/Rayleigh/dataset/UNIFORM_SNR_SAMPLES_training_0-20_SNR_50_BETA.npz",
    #    "val": "/content/drive/MyDrive/GitHub/Rayleigh/dataset/UNIFORM_SNR_SAMPLES_validation_0-20_SNR_50_BETA.npz",
    #},
    "11-15": {
        "train": "/content/drive/MyDrive/GitHub/Rayleigh/dataset/training_11-15_SNR_50_BETA.npz",
        "val": "/content/drive/MyDrive/GitHub/Rayleigh/dataset/validation_11-15_SNR_50_BETA.npz",
    }
}

# Directory salvataggio modelli
save_dir = "/content/drive/MyDrive/GitHub/AWGN/trained_models/benchmarks"
os.makedirs(save_dir, exist_ok=True)

# Load the datasets.

In [ ]:
def load_dataset(filepath):
    """
    Carica i dati X e y da un file .npz.
    Si aspetta che il file contenga 'X_train' o 'X_val' e 'y_train' o 'y_val'.
    """
    data = np.load(filepath)

    # Controlla se le chiavi per il training sono presenti
    if 'X_train' in data and 'y_train' in data:
        print(f"  Caricato Training data da: {filepath}")
        return data['X_train'], data['y_train']
    # Altrimenti, controlla se le chiavi per la validation sono presenti
    elif 'X_val' in data and 'y_val' in data:
        print(f"  Caricato Validation data da: {filepath}")
        return data['X_val'], data['y_val']
    else:
        # Se nessuna delle combinazioni attese è trovata, solleva un errore
        raise ValueError(f"Il file {filepath} non contiene i dati X e y attesi (né 'X_train'/'y_train' né 'X_val'/'y_val'). "
                         f"Chiavi trovate: {list(data.keys())}")

# Benchmark models definition

In [ ]:
# Funzioni modello

def get_tdnn_model(input_shape, model_name='TDNN OFDM-DCSK'):
    # Dal paper: "Reliable and Secure Deep Learning-Based OFDM-DCSK Transceiver Design Without Delivery of Reference Chaotic Sequences"
    inputs = Input(shape=input_shape)
    input_length = input_shape[0]
    x = layers.Reshape((input_length, 1), name="reshape_for_conv")(inputs)
    x = layers.Conv1D(4, 2, activation='relu', padding='same', name="tdnn_conv1")(x)
    x = layers.Conv1D(8, 4, activation='relu', padding='same', name="tdnn_conv2")(x)
    x = layers.Conv1D(16, 8, activation='relu', padding='same', name="tdnn_conv3")(x)
    x = layers.Bidirectional(layers.LSTM(64, return_sequences=False), name="bidirectional_lstm")(x)
    x = layers.Dense(96, name="fc1")(x)
    x = layers.BatchNormalization(name="batch_norm")(x)
    x = layers.Activation('relu', name="relu_fc1")(x)
    outputs = layers.Dense(2, activation='softmax', name="output_softmax")(x)
    model = Model(inputs, outputs, name='DNN_Aided_Demodulator')
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

def get_mc_dlcs_model(input_shape, model_name='MC-DLCSK'):
    # Dal paper: "A Multi-Carrier Deep Learning CSK Communication System"
    if input_shape[0] % 2 != 0:
        raise ValueError("input_shape deve essere pari per MC-DLCSK")
    inputs = Input(shape=input_shape)
    x = layers.Reshape((input_shape[0], 1), name="reshape_for_lstm")(inputs)
    x = layers.Bidirectional(layers.LSTM(5, return_sequences=False), name="bidirectional_lstm")(x)
    x = layers.Dense(2, activation='relu', name="fc1_relu")(x)
    outputs = layers.Dense(2, activation='softmax', name="output_softmax")(x)
    model = Model(inputs, outputs, name=model_name)
    model.compile(optimizer=tf.keras.optimizers.Adam(0.01), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

def get_lstm_dnn_model(input_shape, model_name='LSTM_DNN_OFDM_DCSK'):
    # Dal paper: "Intelligent and Reliable Deep Learning LSTM Neural Networks-Based OFDM-DCSK Demodulation Design"
    inputs = Input(shape=input_shape)
    if len(input_shape) == 1:
        x = layers.Reshape((1, input_shape[0]))(inputs)
    elif len(input_shape) == 2:
        x = inputs
    else:
        raise ValueError("input_shape deve essere 1D o 2D")
    # For K=32 (max length in the paper) authors set 77 LSTM units. In Table 1 69 units for the first Dense Layer.
    x = layers.LSTM(19)(x)
    x = layers.Dense(69, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    outputs = layers.Dense(2, activation='softmax')(x)
    model = Model(inputs, outputs, name=model_name)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

def get_two_conv1d_global_avg_pool_attention_model(input_shape, model_name='TwoConv1D_GlobalAvgPool_Attention'):
    inputs = Input(shape=input_shape)
    x = layers.Reshape((input_shape[0], 1), name="reshape_for_conv")(inputs)
    x = layers.Conv1D(filters=32, kernel_size=3, activation='relu', padding='same', name="conv1d_layer_1")(x)
    conv_output_2 = layers.Conv1D(filters=64, kernel_size=3, activation='relu', padding='same', name="conv1d_layer_2")(x)

    attention_weights = layers.Conv1D(filters=1, kernel_size=1, activation='sigmoid', name="attention_weights")(conv_output_2)
    attended_features = layers.Multiply(name="multiply_attention")([conv_output_2, attention_weights])

    x = layers.GlobalAveragePooling1D(name="global_average_pooling_layer")(attended_features)
    x = layers.Dropout(0.3, name="dropout_layer")(x)
    x = layers.Dense(128, activation='relu', name="fc1")(x)
    outputs = layers.Dense(2, activation='softmax', name="output_softmax")(x)
    model = Model(inputs, outputs, name=model_name)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model


models_dict = {
    #"TDNN OFDM-DCSK": get_tdnn_model,
    "Ultra-CANv3": get_two_conv1d_global_avg_pool_attention_model,
    #"LSTM_DNN_OFDM_DCSK": get_lstm_dnn_model
}

# Training...

In [ ]:
# Addestramento
for snr_range, datasets in paths.items():
    x_train, y_train = load_dataset(datasets['train'])
    x_val, y_val = load_dataset(datasets['val'])

    for model_name, builder in models_dict.items():
        print(f"\nAddestramento modello: {model_name} | SNR range: {snr_range}")
        model = builder(input_shape=x_train.shape[1:])

        callback = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

        start_time = time.time() # Registra il tempo di inizio training

        # Accordint to the papers...
        if model_name=="MC-DLCSK":
          # As recommended in the original paper.
          epochs_nr = 20
        else:
          epochs_nr = 30

        if model_name=="LSTM_DNN_OFDM_DCSK":
          batch = 20
        else:
          batch = 50

        model.fit(
              x_train, y_train,
              validation_data=(x_val, y_val),
              epochs=epochs_nr,
              batch_size=batch,
              callbacks=[callback],
              verbose=1
          )



        end_time = time.time() # Registra il tempo di fine training
        total_training_time = end_time - start_time

        print(f"Training completato.")
        print(f"Tempo totale di training: {total_training_time:.2f} secondi.") # Stampa il tempo totale

        save_path = os.path.join(save_dir, f"{model_name.replace(' ', '_')}-ORIGINAL-50-BETA_rayleigh_snr_{snr_range}.h5")
        model.save(save_path)
        print(f"Salvato modello in: {save_path}")

  Caricato Training data da: /content/drive/MyDrive/GitHub/Rayleigh/dataset/training_11-15_SNR_50_BETA.npz
  Caricato Validation data da: /content/drive/MyDrive/GitHub/Rayleigh/dataset/validation_11-15_SNR_50_BETA.npz

Addestramento modello: MC-DLCSK | SNR range: 11-15
Epoch 1/20
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 21s 15ms/step - accuracy: 0.5881 - loss: 0.6663 - val_accuracy: 0.6194 - val_loss: 0.6233
Epoch 2/20
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 17s 12ms/step - accuracy: 0.5684 - loss: 0.6738 - val_accuracy: 0.6482 - val_loss: 0.6321
Epoch 3/20
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 15s 13ms/step - accuracy: 0.7126 - loss: 0.5377 - val_accuracy: 0.9758 - val_loss: 0.0734
Epoch 4/20
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 20s 13ms/step - accuracy: 0.9727 - loss: 0.0739 - val_accuracy: 0.9793 - val_loss: 0.0529
Epoch 5/20
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 15s 12ms/step - accuracy: 0.9796 - loss: 0.0527 - val_accuracy: 0.9786 - val_loss: 0.0568
Epoch 6/20
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 14s 12ms/step - accuracy: 0.

Training completato.
Tempo totale di training: 356.32 secondi.
Salvato modello in: /content/drive/MyDrive/GitHub/AWGN/trained_models/benchmarks/MC-DLCSK-ORIGINAL-50-BETA_rayleigh_snr_11-15.h5


In [ ]:
# Addestramento
for snr_range, datasets in paths.items():
    x_train, y_train = load_dataset(datasets['train'])
    x_val, y_val = load_dataset(datasets['val'])

    for model_name, builder in models_dict.items():
        print(f"\nAddestramento modello: {model_name} | SNR range: {snr_range}")
        model = builder(input_shape=x_train.shape[1:])

        callback = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

        start_time = time.time() # Registra il tempo di inizio training

        # Accordint to the papers...
        if model_name=="MC-DLCSK":
          # As recommended in the original paper.
          epochs_nr = 20
        else:
          epochs_nr = 30

        if model_name=="LSTM_DNN_OFDM_DCSK":
          batch = 20
        else:
          batch = 50

        model.fit(
              x_train, y_train,
              validation_data=(x_val, y_val),
              epochs=epochs_nr,
              batch_size=batch,
              callbacks=[callback],
              verbose=1
          )



        end_time = time.time() # Registra il tempo di fine training
        total_training_time = end_time - start_time

        print(f"Training completato.")
        print(f"Tempo totale di training: {total_training_time:.2f} secondi.") # Stampa il tempo totale

        save_path = os.path.join(save_dir, f"{model_name.replace(' ', '_')}-ORIGINAL-50-BETA_rayleigh_snr_{snr_range}.h5")
        model.save(save_path)
        print(f"Salvato modello in: {save_path}")

  Caricato Training data da: /content/drive/MyDrive/GitHub/Rayleigh/dataset/training_11-15_SNR_50_BETA.npz
  Caricato Validation data da: /content/drive/MyDrive/GitHub/Rayleigh/dataset/validation_11-15_SNR_50_BETA.npz

Addestramento modello: Ultra-CANv3 | SNR range: 11-15
Epoch 1/30
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 12s 7ms/step - accuracy: 0.8345 - loss: 0.3249 - val_accuracy: 0.9766 - val_loss: 0.0565
Epoch 2/30
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 9s 7ms/step - accuracy: 0.9714 - loss: 0.0680 - val_accuracy: 0.9780 - val_loss: 0.0514
Epoch 3/30
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 7s 6ms/step - accuracy: 0.9741 - loss: 0.0615 - val_accuracy: 0.9790 - val_loss: 0.0499
Epoch 4/30
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 11s 6ms/step - accuracy: 0.9747 - loss: 0.0609 - val_accuracy: 0.9787 - val_loss: 0.0474
Epoch 5/30
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 9s 7ms/step - accuracy: 0.9752 - loss: 0.0572 - val_accuracy: 0.9786 - val_loss: 0.0486
Epoch 6/30
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 10s 7ms/step - accuracy: 0.9769 -

Training completato.
Tempo totale di training: 290.59 secondi.
Salvato modello in: /content/drive/MyDrive/GitHub/AWGN/trained_models/benchmarks/Ultra-CANv3-ORIGINAL-50-BETA_rayleigh_snr_11-15.h5
